In This Notebook we find our Model used but before we do the fucntion of data cleaning and preprocessing then we run the models and upload them to our mlflow Server

In [22]:
import os
import pandas as pd
import mlflow
from dotenv import load_dotenv
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from prep import transform_data_f, clean_data2 , transform_data_f_2_c
from MlProc1 import RandomForestModel, XGBoostModel, logisticRegressionModel, RandomForestModelG, XGBoostModelG, logisticRegressionModelG, SVMModelG




In [17]:
# Load environment variables (DagsHub credentials) from .env file
load_dotenv("E:\\cycle_3eme\\mlops\\projet_clone\\MLOps-Training\\backend\\src\\.env")


True

In [5]:
# Get DagsHub credentials from the environment
DagsHub_username = os.getenv("DagsHub_username")
DagsHub_token = os.getenv("DagsHub_token")

# Check if credentials are loaded
if DagsHub_username is None or DagsHub_token is None:
    print("Error: Could not load DagsHub credentials from .env file.")
else:
    print(f"DagsHub username and token are loaded successfully.")

# Set DagsHub credentials for MLflow
os.environ['MLFLOW_TRACKING_USERNAME'] = DagsHub_username
os.environ["MLFLOW_TRACKING_PASSWORD"] = DagsHub_token

# Setup mlflow
mlflow.set_tracking_uri('https://dagshub.com/aymenalimii4070/Ml_OPS_Movies.mlflow')
mlflow.set_experiment("Movies-Rating-experiment-3")

# Data URL and version
version = "v3.with_3_clasess-with_grid_search_with_2_Classes"
data_url = "E:\\cycle_3eme\\mlops\\projet_clone\\MLOps-Training\\backend\\src\\data_set\\movies.csv"




DagsHub username and token are loaded successfully.


In [18]:
df = pd.read_csv(data_url)
print(df.head(5))


                                MOVIES         YEAR  \
0                        Blood Red Sky       (2021)   
1  Masters of the Universe: Revelation     (2021– )   
2                     The Walking Dead  (2010–2022)   
3                       Rick and Morty     (2013– )   
4                      Army of Thieves       (2021)   

                                        GENRE  RATING  \
0      \nAction, Horror, Thriller                 6.1   
1  \nAnimation, Action, Adventure                 5.0   
2       \nDrama, Horror, Thriller                 8.2   
3  \nAnimation, Adventure, Comedy                 9.2   
4         \nAction, Crime, Horror                 NaN   

                                            ONE-LINE  \
0  \nA woman with a mysterious illness is forced ...   
1  \nThe war for Eternia begins again in what may...   
2  \nSheriff Deputy Rick Grimes wakes up from a c...   
3  \nAn animated series that follows the exploits...   
4  \nA prequel, set before the events of Army 

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB
None


In [20]:
# Cleaning
df_cleaned=clean_data2(df)
# preprocessing
display(df_cleaned)

,movies,rating,one-line,votes,gross,director,stars_only,action,adventure,animation,...,content_Standard,content_Long,content_Très long,content_Movie,content_TV Movie,content_TV Short,content_TV Special,content_Unknown,content_Video,content_Video Game
0,2.182977,6.10,6.454030,21062.000000,0.0,2458,4877,1,0,0,...,False,False,True,True,False,False,False,False,False,False
1,4.315215,5.00,7.034649,17870.000000,0.0,2315,1153,1,1,1,...,False,False,False,True,False,False,False,False,False,False
2,6.780270,8.20,6.742347,885805.000000,0.0,2315,406,0,0,0,...,False,False,False,True,False,False,False,False,False,False
3,9.353854,9.20,7.080458,414849.000000,0.0,2315,3273,0,1,1,...,False,False,False,True,False,False,False,False,False,False
4,1.703747,5.80,6.963683,90390.060000,0.0,2074,4198,1,0,0,...,False,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9563,6.297324,6.55,6.997828,931.898750,0.0,2288,1951,0,0,0,...,False,True,False,True,False,False,False,False,False,False
9564,8.279235,7.80,6.997828,104.001762,0.0,2315,4681,1,1,1,...,False,False,False,True,False,False,False,False,False,False
9565,4.514454,7.65,6.997828,111.625000,0.0,2345,4681,0,0,0,...,False,False,False,True,False,False,False,False,False,False
9566,3.353738,6.85,6.997828,3388.459000,0.0,1583,4473,0,1,0,...,False,False,False,True,False,False,False,False,False,False


In [9]:
X_train, X_test, y_train, y_test=transform_data_f(df_cleaned)

2 Clases transformation :

In [25]:
#Data with only 2 Classes
X_train, X_test, y_train, y_test=transform_data_f_2_c(df_cleaned)

TypeError: cannot unpack non-iterable NoneType object

In [10]:
print(y_train.value_counts())
print("/////////////")
print(y_test.value_counts())


rating
2    5007
1    2587
0      60
Name: count, dtype: int64


In [15]:
print(X_test.shape)

(1914, 47)


##Random_Forest_Model

In [18]:
RandomForestModel(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)

XGBOOSTModel

In [16]:
XGBoostModel(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)

In [17]:
logisticRegressionModel(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)


In [27]:
print(X_test.shape)


(1914, 47)


Models with the use of grid search to find the best hyperparameters:

In [28]:
logisticRegressionModelG(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)


In [29]:
XGBoostModelG(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)

In [11]:
RandomForestModelG(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)

In [30]:
SVMModelG(data_url,version,df_cleaned,X_train,y_train,X_test,y_test)

: 

: 